In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output
from tqdm.notebook import tqdm

In [51]:
# First date range pickers
start_date_picker1 = widgets.DatePicker(
    description='Start Date 1',
    disabled=False,
    value=datetime(2024, 1, 1)
)

end_date_picker1 = widgets.DatePicker(
    description='End Date 1',
    disabled=False,
    value=datetime(2024, 1, 29)
)

# Second date range pickers
start_date_picker2 = widgets.DatePicker(
    description='Start Date 2',
    disabled=False,
    value=datetime(2024, 2, 1)
)

end_date_picker2 = widgets.DatePicker(
    description='End Date 2',
    disabled=False,
    value=datetime(2024, 2, 29)
)

# Update button (we can keep the same button)
load_data_button = widgets.Button(
    description='Load Data',
    disabled=False,
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to load data',
    icon='download'  # (Optional) FontAwesome icon
)

# Dropdown widget to select a model
model_selector = widgets.Dropdown(
    options=[],
    description='Model:',
    disabled=True,
)

# Button to generate comparison graphs
generate_graphs_button = widgets.Button(
    description='Generate Graphs',
    disabled=True,
    button_style='',
    tooltip='Click to generate comparison graphs',
    icon='bar-chart'  # (Optional) FontAwesome icon
)

# Button to show data used for plotting
show_data_button = widgets.Button(
    description='Show Data',
    disabled=True,
    button_style='',
    tooltip='Click to display data used for plotting',
    icon='table'  # (Optional) FontAwesome icon
)

# Multi-select widget for column selection
column_selector = widgets.SelectMultiple(
    options=[],  # This will be populated once the data is loaded
    description='Columns:',
    disabled=True,
    layout=widgets.Layout(width='50%')
)

In [52]:
# Create an output widget to display the output
output_widget = widgets.Output()

# Output widget to display data
data_output = widgets.Output()

# Output widget to display graphs
graphs_output = widgets.Output()


In [53]:
def load_data(b):
    with output_widget:
        clear_output(wait=True)
        
        # Get the selected dates for both date ranges
        start_date1 = start_date_picker1.value
        end_date1 = end_date_picker1.value
        start_date2 = start_date_picker2.value
        end_date2 = end_date_picker2.value
        
        # Input validation for date ranges
        if None in [start_date1, end_date1, start_date2, end_date2]:
            print("Please select all start and end dates.")
            return
        if start_date1 > end_date1 or start_date2 > end_date2:
            print("Start dates must be before or equal to end dates.")
            return
        
        # Initialize empty lists to store data
        data_list1 = []
        data_list2 = []

        # Define the Columns to Load
        columns_to_load = [
        'date', 'model', 'serial_number',
        'smart_5_normalized', 'smart_5_raw', 'smart_187_normalized', 'smart_187_raw', 'smart_188_normalized', 'smart_188_raw',
        'smart_197_normalized','smart_197_raw', 'smart_198_normalized','smart_198_raw', 'capacity_bytes', 'failure'
        ]
        
        # Define a function to load data for a date range
        def load_data_for_date_range(start_date_dt, end_date_dt, data_list, date_range_label):
            date_range = pd.date_range(start_date_dt, end_date_dt)
            for current_date in tqdm(date_range, desc=f'Loading Data for {date_range_label}'):
                date_str = current_date.strftime('%Y-%m-%d')
                file_name = f"{date_str}.csv"  
                file_path = os.path.join('D:/Backblaze_Data/data_Q2_2024/', file_name)

                if os.path.exists(file_path):
                    try:
                        df = pd.read_csv(file_path, usecols=columns_to_load)
                        df = df[df['failure'] == 1]  # Filter for failed drives
                        if not df.empty:
                            df['date'] = date_str
                            data_list.append(df)
                    except Exception as e:
                        print(f"Error loading {file_name}: {e}")
                else:
                    print(f"File not found for date: {date_str}")
        
        # Convert dates to datetime objects
        start_date_dt1 = pd.to_datetime(start_date1)
        end_date_dt1 = pd.to_datetime(end_date1)
        start_date_dt2 = pd.to_datetime(start_date2)
        end_date_dt2 = pd.to_datetime(end_date2)

        # Load data for both date ranges
        load_data_for_date_range(start_date_dt1, end_date_dt1, data_list1, 'Date Range 1')
        load_data_for_date_range(start_date_dt2, end_date_dt2, data_list2, 'Date Range 2')

        if not data_list1 and not data_list2:
            print("No failed drives found for the selected date ranges.")
            return

        # Concatenate data for both date ranges
        global data_df1, data_df2
        data_df1 = pd.concat(data_list1, ignore_index=True) if data_list1 else pd.DataFrame()
        data_df2 = pd.concat(data_list2, ignore_index=True) if data_list2 else pd.DataFrame()

        # Get the list of models available in both datasets
        models1 = data_df1['model'].unique()
        models2 = data_df2['model'].unique()
        available_models = sorted(list(set(models1) | set(models2)))

        if available_models:
            # Assign data_df1 and data_df2 to all_failed_drives_df1 and all_failed_drives_df2
            all_failed_drives_df1 = data_df1.copy()
            all_failed_drives_df2 = data_df2.copy()
                        
            # Group by model and count the number of failed drives
            failed_drives_by_model_1 = all_failed_drives_df1.groupby('model')['serial_number'].nunique()
            failed_drives_by_model_2 = all_failed_drives_df2.groupby('model')['serial_number'].nunique()
            print("Failed Drives Grouped by Model for date range 1:")
            display(failed_drives_by_model_1)
            print("Failed Drives Grouped by Model for date range 2:")
            display(failed_drives_by_model_2)

            # Update the column selector with available columns from the data
            common_columns = sorted(list(set(data_df1.columns) & set(data_df2.columns)))
            column_selector.options = common_columns  # Set available columns
            column_selector.disabled = False  # Enable column selector
            
            # Update the model selector options
            model_selector.options = available_models
            model_selector.disabled = False
            generate_graphs_button.disabled = False
            show_data_button.disabled = False
            
            print("Data loaded successfully. Please select a model and click 'Generate Graphs'.")
        else:
            print("No models found in the data.")
            model_selector.options = []
            model_selector.disabled = True
            generate_graphs_button.disabled = True
            show_data_button.disabled = True
        


In [54]:
def show_data(b):
    with data_output:
        clear_output(wait=True)
        selected_model = model_selector.value
        selected_columns = list(column_selector.value)  # Get the selected columns
        if not selected_model:
            print("Please select a model to display data.")
            return
        
        if not selected_columns:
            print("Please select at least one column to display.")
            return
        
        # Use the already loaded data
        # Filter data for the selected model
        model_data1 = data_df1[data_df1['model'] == selected_model]
        model_data2 = data_df2[data_df2['model'] == selected_model]
        
        if model_data1.empty and model_data2.empty:
            print(f"No data found for model {selected_model} in both date ranges.")
            return
        
        # Display data
        print(f"Data for model {selected_model} in Date Range 1:")
        display(model_data1[selected_columns] if not model_data1.empty else pd.DataFrame())
        print(f"Data for model {selected_model} in Date Range 2:")
        display(model_data2[selected_columns] if not model_data2.empty else pd.DataFrame())



In [55]:
def generate_kde_plots(b):
    """
    Generates and displays overlaid Kernel Density Estimate (KDE) plots for 
    each SMART attribute for the selected model in the two date ranges.
    """
    with graphs_output:
        clear_output(wait=True)
        selected_model = model_selector.value
        if not selected_model:
            print("Please select a model.")
            return
        
        # Filter data for the selected model
        model_data1 = data_df1[data_df1['model'] == selected_model]
        model_data2 = data_df2[data_df2['model'] == selected_model]

        if model_data1.empty and model_data2.empty:
            print(f"No failed drives found for model {selected_model} in both date ranges.")
            return
        
        # SMART attributes to compare
        attributes = [
            'smart_5_normalized', 'smart_5_raw', 'smart_187_normalized', 'smart_187_raw',
            'smart_188_normalized', 'smart_188_raw', 'smart_197_normalized', 'smart_197_raw', 
            'smart_198_normalized', 'smart_198_raw'
        ]
        
        # Plot overlaid KDE for each attribute
        for attr in attributes:
            # Check if the attribute exists in both datasets and is not completely NaN
            attr_in_data1 = attr in model_data1.columns and not model_data1[attr].isnull().all()
            attr_in_data2 = attr in model_data2.columns and not model_data2[attr].isnull().all()

            if not attr_in_data1 and not attr_in_data2:
                print(f"Attribute {attr} is not available in both date ranges. Skipping.")
                continue  # Skip this attribute if it's not available in both datasets

            # Prepare data for plotting
            data_to_plot = []
            labels = []
            
            if attr_in_data1:
                data_to_plot.append(model_data1[attr].dropna())
                labels.append('Date Range 1')
                
            if attr_in_data2:
                data_to_plot.append(model_data2[attr].dropna())
                labels.append('Date Range 2')

            if data_to_plot:
                # Overlaid KDE plot
                plt.figure(figsize=(10, 6))
                
                for data, label in zip(data_to_plot, labels):
                    sns.kdeplot(data, label=label, fill=True)  # Updated 'fill=True' instead of 'shade=True'
                
                plt.title(f'Overlaid KDE for {attr} ({selected_model})')
                plt.xlabel(attr)
                plt.ylabel('Density')
                plt.legend()
                plt.tight_layout()
                plt.show()


In [57]:
# Link the Show Data button to the function
show_data_button.on_click(show_data)

# Link the button to the function
load_data_button.on_click(load_data)

# Link the KDE plot generation to a new button or the existing graph generation button
generate_graphs_button.on_click(generate_kde_plots)

In [58]:
# UI layout
ui = widgets.VBox([
    widgets.HBox([start_date_picker1, end_date_picker1]),
    widgets.HBox([start_date_picker2, end_date_picker2]),
    load_data_button,
    output_widget,
    model_selector,
    column_selector,  # Added column selector here
    widgets.HBox([generate_graphs_button, show_data_button]),
    graphs_output,
    data_output
])

In [ ]:
display(ui)

In [60]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output
from tqdm.notebook import tqdm
import gc

# Date range pickers
start_date_picker = widgets.DatePicker(
    description='Start Date',
    disabled=False,
    value=datetime(2024, 1, 1)
)

end_date_picker = widgets.DatePicker(
    description='End Date',
    disabled=False,
    value=datetime(2024, 6, 30)
)

# Button to load data
load_data_button = widgets.Button(
    description='Load Data',
    disabled=False,
    button_style='',
    tooltip='Click to load data',
    icon='download'
)

# Output widget to display messages
output_widget = widgets.Output()

# Multi-select widget to select SMART attributes
smart_attributes_list = [
    'smart_5_normalized', 'smart_5_raw',
    'smart_187_normalized', 'smart_187_raw',
    'smart_188_normalized', 'smart_188_raw',
    'smart_197_normalized', 'smart_197_raw',
    'smart_198_normalized', 'smart_198_raw',
    'capacity_bytes'
]

attribute_selector = widgets.SelectMultiple(
    options=smart_attributes_list,
    description='SMART Attributes:',
    disabled=True  # Initially disabled
)

# Button to perform correlation analysis
correlation_button = widgets.Button(
    description='Correlation Analysis',
    disabled=True,
    button_style='',
    tooltip='Click to perform correlation analysis',
    icon='chart-line'
)

# Output widget to display correlation results
correlation_output = widgets.Output()

def load_data(b):
    with output_widget:
        clear_output(wait=True)
        
        # Get the selected dates for the analysis
        start_date = start_date_picker.value
        end_date = end_date_picker.value
        
        # Input validation for date ranges
        if None in [start_date, end_date]:
            print("Please select start and end dates.")
            return
        if start_date > end_date:
            print("Start date must be before or equal to end date.")
            return
        
        # Initialize empty list to store data
        data_list = []

        # Define the Columns to Load
        columns_to_load = [
            'date', 'model', 'serial_number', 'failure',
            'smart_5_normalized', 'smart_5_raw',
            'smart_187_normalized', 'smart_187_raw',
            'smart_188_normalized', 'smart_188_raw',
            'smart_197_normalized', 'smart_197_raw',
            'smart_198_normalized', 'smart_198_raw',
            'capacity_bytes'
        ]
        
        # Load data for the date range
        date_range = pd.date_range(start_date, end_date)
        for current_date in tqdm(date_range, desc='Loading Data'):
            date_str = current_date.strftime('%Y-%m-%d')
            file_name = f"{date_str}.csv"  
            file_path = os.path.join('D:/Backblaze_Data/data_Q2_2024/', file_name)

            if os.path.exists(file_path):
                try:
                    df = pd.read_csv(file_path, usecols=columns_to_load)
                    df['date'] = date_str
                    data_list.append(df)
                except Exception as e:
                    print(f"Error loading {file_name}: {e}")
            else:
                print(f"File not found for date: {date_str}")
        
        if not data_list:
            print("No data found for the selected date range.")
            return
        
        # Concatenate data
        global data_df
        data_df = pd.concat(data_list, ignore_index=True)
        
        # Clean up to free memory
        del data_list
        gc.collect()
        
        # Enable the correlation analysis button and attribute selector
        correlation_button.disabled = False
        attribute_selector.disabled = False
        print("Data loaded successfully. Please select SMART attributes and click 'Correlation Analysis'.")

def calculate_drive_lifetime(data_df):
    # Convert 'date' column to datetime
    data_df['date'] = pd.to_datetime(data_df['date'])
    
    # Identify failed drives
    failed_drives = data_df[data_df['failure'] == 1]['serial_number'].unique()
    
    # Filter data for failed drives
    failed_drives_data = data_df[data_df['serial_number'].isin(failed_drives)]
    
    # Calculate first and last appearance dates for each drive
    lifetime_df = failed_drives_data.groupby('serial_number').agg({
        'date': ['min', 'max']
    }).reset_index()
    
    # Flatten MultiIndex columns
    lifetime_df.columns = ['serial_number', 'first_date', 'last_date']
    
    # Calculate lifetime in days
    lifetime_df['lifetime_days'] = (lifetime_df['last_date'] - lifetime_df['first_date']).dt.days + 1
    
    return lifetime_df

def aggregate_smart_attributes(data_df, selected_attributes):
    # Filter data for failed drives
    failed_drives_data = data_df[data_df['failure'] == 1]
    
    # Group by 'serial_number' and compute aggregate statistics
    # Using mean for simplicity
    smart_aggregated = failed_drives_data.groupby('serial_number')[selected_attributes].mean().reset_index()
    
    return smart_aggregated

def perform_correlation_analysis(b):
    with correlation_output:
        clear_output(wait=True)
        
        # Get selected SMART attributes
        selected_attributes = list(attribute_selector.value)
        
        if not selected_attributes:
            print("Please select at least one SMART attribute.")
            return
        
        # Calculate drive lifetime
        lifetime_df = calculate_drive_lifetime(data_df)
        
        if lifetime_df.empty:
            print("No failed drives found in the data.")
            return
        
        # Aggregate SMART attributes
        smart_aggregated = aggregate_smart_attributes(data_df, selected_attributes)
        
        # Merge lifetime data with SMART attributes
        analysis_df = pd.merge(lifetime_df, smart_aggregated, on='serial_number', how='inner')
        
        # Prepare data for correlation
        correlation_columns = ['lifetime_days'] + selected_attributes
        correlation_data = analysis_df[correlation_columns].dropna()
        
        if correlation_data.empty:
            print("No data available for correlation analysis after removing missing values.")
            return
        
        # Calculate Spearman correlation
        correlation_matrix = correlation_data.corr(method='spearman')
        
        # Display correlation matrix
        print("Spearman Correlation Matrix:")
        display(correlation_matrix)
        
        # Plot correlation heatmap
        plt.figure(figsize=(8, 6))
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
        plt.title('Spearman Correlation between SMART Attributes and Drive Lifetime')
        plt.show()
        
        # Plot scatter plots for each SMART attribute vs lifetime
        for attr in selected_attributes:
            plt.figure(figsize=(6, 4))
            sns.scatterplot(data=correlation_data, x=attr, y='lifetime_days')
            plt.title(f'Lifetime vs {attr}')
            plt.xlabel(attr)
            plt.ylabel('Lifetime (days)')
            plt.show()

# Link the buttons to the functions
load_data_button.on_click(load_data)
correlation_button.on_click(perform_correlation_analysis)

# Arrange the widgets in the UI
ui = widgets.VBox([
    widgets.HBox([start_date_picker, end_date_picker]),
    load_data_button,
    output_widget,
    attribute_selector,
    correlation_button,
    correlation_output
])

# Display the UI
display(ui)


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output
from tqdm.notebook import tqdm

In [2]:
# Date picker widgets for selecting start and end dates
start_date_picker = widgets.DatePicker(
    description='Start Date',
    disabled=False,
    value=datetime(2024, 4, 1)
)

end_date_picker = widgets.DatePicker(
    description='End Date',
    disabled=False,
    value=datetime(2024, 4, 30)
)

In [3]:
# Dropdown widget to select a model
model_selector = widgets.Dropdown(
    options=[],
    description='Model:',
    disabled=True,
)

# Slider to select duration (10 to 30 days)
duration_slider = widgets.IntSlider(
    value=10,
    min=10,
    max=30,
    step=1,
    description='Duration (days):',
    disabled=True,
)


In [4]:
# Button to load data based on the selected date range
load_data_button = widgets.Button(
    description='Load Data',
    disabled=False,
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to load data',
    icon='download'  # (Optional) FontAwesome icon
)

In [6]:
# Dropdown widget to select a drive (serial number)
drive_selector = widgets.Dropdown(
    options=[],
    description='Drive (SN):',
    disabled=True,
)

# Multi-select widget to select SMART attributes
smart_attributes_list = ['smart_5_normalized', 'smart_187_normalized', 'smart_188_normalized', 'smart_197_normalized', 'smart_198_normalized']
smart_attribute_selector = widgets.SelectMultiple(
    options=smart_attributes_list,
    value=[smart_attributes_list[0]],
    description='SMART Attributes:',
    disabled=True,
)


In [7]:
# Button to generate graphs
generate_graphs_button = widgets.Button(
    description='Generate Graphs',
    disabled=False,
    button_style='',
    tooltip='Click to generate graphs',
    icon='line-chart'  # FontAwesome icon 4.7 
)

In [8]:
# Button to check drive info
check_info_button = widgets.Button(
    description='Check Information',
    disabled=False,
    button_style='',
    tooltip='Click to check information',
    icon='info-circle'  # FontAwesome icon
)

In [9]:
# Output widget to display messages and lists
output = widgets.Output()

In [10]:
def load_data(b):
    with output:
        clear_output(wait=True)
        start_date = start_date_picker.value
        end_date = end_date_picker.value
        
        if start_date is None or end_date is None:
            print("Please select both start and end dates.")
            return
        
        if start_date > end_date:
            print("Start date must be before or equal to end date.")
            return
        
        # Initialize an empty list to store failed drives information
        failed_drives_list = []
        
        # Convert dates to datetime objects
        start_date_dt = pd.to_datetime(start_date)
        end_date_dt = pd.to_datetime(end_date)
        
        # Generate a list of dates
        date_range = pd.date_range(start_date_dt, end_date_dt)
        
        # Load data files and collect failed drives
        for current_date in tqdm(date_range, desc='Loading Data'):
            date_str = current_date.strftime('%Y-%m-%d')
            file_name = f"{date_str}.csv"
            file_path = os.path.join('D:/Backblaze_Data/data_Q2_2024/', file_name) 
            
            if os.path.exists(file_path):
                df = pd.read_csv(file_path)
                failed_drives = df[df['failure'] == 1]
                if not failed_drives.empty:
                    failed_drives = failed_drives.copy()  # Make a copy to avoid SettingWithCopyWarning
                    failed_drives['date'] = date_str
                    failed_drives_list.append(failed_drives)
            else:
                print(f"File not found for date: {date_str}")
        
        if failed_drives_list:
            # Concatenate all failed drives dataframes into one
            global all_failed_drives_df
            all_failed_drives_df = pd.concat(failed_drives_list, ignore_index=True)
            
            # Group by model and count the number of failed drives
            failed_drives_by_model = all_failed_drives_df.groupby('model')['serial_number'].nunique()
            print("Failed Drives Grouped by Model:")
            display(failed_drives_by_model)
            
            # Update the model selection options
            model_selector.options = failed_drives_by_model.index.tolist()
            
            # Enable the model selector and duration slider
            model_selector.disabled = False
            duration_slider.disabled = False
        else:
            print("No failed drives found in the selected date range.")
            model_selector.options = []
            model_selector.disabled = True
            duration_slider.disabled = True


In [11]:
load_data_button.on_click(load_data)

In [39]:
def update_drive_list(change):
    with output:
        # Get the selected model
        selected_model = model_selector.value
        if selected_model:
            # Filter the DataFrame for the selected model
            model_failed_drives = all_failed_drives_df[all_failed_drives_df['model'] == selected_model]
            # Get the unique serial numbers
            serial_numbers = model_failed_drives['serial_number'].unique()
            # Update the drive selector options
            drive_selector.options = serial_numbers.tolist()
            drive_selector.disabled = False
            smart_attribute_selector.disabled = False
        else:
            drive_selector.options = []
            drive_selector.disabled = True
            smart_attribute_selector.disabled = True

In [40]:
model_selector.observe(update_drive_list, names='value')

In [41]:
graphs_output = widgets.Output()

In [42]:
infos_output = widgets.Output()

In [43]:
def generate_graphs(b):
    with graphs_output:
        clear_output(wait=True)
        selected_model = model_selector.value
        selected_drive = drive_selector.value
        selected_attributes = list(smart_attribute_selector.value)
        duration_days = duration_slider.value
        
        if not selected_model or not selected_drive or not selected_attributes:
            print("Please select a model, drive, and at least one SMART attribute.")
            return
        
        # Get the data for the selected drive
        drive_data = []
        # Calculate the date range based on the selected duration
        end_date_dt = pd.to_datetime(all_failed_drives_df['date'].max())
        start_date_dt = end_date_dt - timedelta(days=duration_days - 1)
        
        # Generate date range
        date_range = pd.date_range(start_date_dt, end_date_dt)
        
        # Load data for the selected drive over the date range
        for current_date in tqdm(date_range, desc='Loading Data'):
            date_str = current_date.strftime('%Y-%m-%d')
            file_name = f"{date_str}.csv"  
            file_path = os.path.join('D:/Backblaze_Data/data_Q2_2024/', file_name)  
            
            if os.path.exists(file_path):
                df = pd.read_csv(file_path)
                # Filter for the selected drive
                drive_row = df[df['serial_number'] == selected_drive]
                if not drive_row.empty:
                    drive_row = drive_row.copy()
                    drive_row['date'] = date_str
                    drive_data.append(drive_row)
            else:
                print(f"File not found for date: {date_str}")
        
        if drive_data:
            drive_df = pd.concat(drive_data, ignore_index=True)
            drive_df['date'] = pd.to_datetime(drive_df['date'])
            drive_df = drive_df.sort_values('date')
            
            # Plot the selected SMART attributes
            for attr in selected_attributes:
                if attr in drive_df.columns:
                    plt.figure(figsize=(12, 6))
                    plt.plot(drive_df['date'], drive_df[attr], marker='o')
                    plt.xlabel('Date')
                    plt.ylabel(attr)
                    plt.title(f'{attr} Over Time for Drive {selected_drive}')
                    plt.grid(True)
                    plt.show()
                else:
                    print(f"Attribute {attr} not found in data.")
        else:
            print("No data found for the selected drive over the specified duration.")


In [44]:
def check_info(b):
    with infos_output:
        clear_output(wait=True)
        selected_model = model_selector.value
        selected_drive = drive_selector.value
                
        if not selected_model or not selected_drive:
            print("Please select a drive")
            return
        failure_data = all_failed_drives_df[all_failed_drives_df['serial_number'] == selected_drive]
        if not failure_data.empty:
            failure_dates = failure_data['date'].unique()
            print(f"Drive {selected_drive} failed on the following date: {failure_dates}")
        else:
            print("No failure information found for the selected drive.")

In [45]:
generate_graphs_button.on_click(generate_graphs)

In [46]:
check_info_button.on_click(check_info)

In [47]:
# Arrange the widgets in the UI
ui = widgets.VBox([
    widgets.HBox([start_date_picker, end_date_picker, load_data_button]),
    output,
    widgets.HBox([model_selector, duration_slider]),
    widgets.HBox([drive_selector, smart_attribute_selector]),  
    widgets.HBox([check_info_button, generate_graphs_button]),
    infos_output,
    graphs_output
])

In [48]:
display(ui)
